In [ ]:
import pandas as pd
import numpy as np

# New Metadata Analysis

In [ ]:
import os

In [ ]:
plates = pd.read_csv("../data/metadata/plate.csv.gz")
wells = pd.read_csv("../data/metadata/well.csv.gz")
compound = pd.read_csv("../data/metadata/compound.csv.gz")
crispr = pd.read_csv("../data/metadata/crispr.csv.gz")
orf = pd.read_csv("../data/metadata/orf.csv.gz")

In [ ]:
jump_moa = pd.read_csv("../data/metadata/JUMP-MOA_compound_metadata.tsv",sep='\t')
jump_target1_compound = pd.read_csv("../data/metadata/JUMP-Target-1_compound_metadata.tsv",sep='\t')
jump_target1_orf= pd.read_csv("../data/metadata/JUMP-Target-1_orf_metadata.tsv",sep='\t')
jump_target1_crispr = pd.read_csv("../data/metadata/JUMP-Target-1_crispr_metadata.tsv",sep='\t')
jump_target2 = pd.read_csv("../data/metadata/JUMP-Target-2_compound_metadata.tsv",sep='\t')

In [ ]:
plates.Metadata_PlateType.unique()

In [ ]:
plates.query('Metadata_PlateType=="TARGET2"').groupby('Metadata_Source').size().sort_values()

In [ ]:
plates.query('Metadata_PlateType=="TARGET1"').groupby('Metadata_Source').size().sort_values()

In [ ]:
plates.query('Metadata_PlateType=="CRISPR"').groupby('Metadata_Source').size().sort_values()

In [ ]:
plates.query('Metadata_PlateType=="ORF"').groupby('Metadata_Source').size().sort_values()

In [ ]:
plates.query('Metadata_PlateType=="DMSO"').groupby('Metadata_Source').size().sort_values()

## Analysis of content

In [ ]:
plates.head()

In [ ]:
wells.head()

In [ ]:
compound.head()

In [ ]:
crispr.head()

In [ ]:
orf.head()

### Merging

In [ ]:
meta = pd.merge(plates.dropna(),wells.dropna(),how='inner',on=['Metadata_Source','Metadata_Plate'])

In [ ]:
meta_compound = pd.merge(meta.dropna(),compound.dropna(),how='inner',on=['Metadata_JCP2022'])

In [ ]:
meta_compound.columns

In [ ]:
meta_orf = pd.merge(meta.dropna(),orf.dropna(),how='inner',on=['Metadata_JCP2022'])

In [ ]:
meta_orf.columns

In [ ]:
meta_crispr = pd.merge(meta.dropna(),crispr.dropna(),how='inner',on=['Metadata_JCP2022'])

In [ ]:
meta_crispr.columns

### Analysis

In [ ]:
pd.merge(meta_compound,jump_moa.dropna(subset=['InChIKey']),left_on=['Metadata_InChIKey'],right_on=['InChIKey'],how='inner')

## jump_moa

In [ ]:
counts = pd.merge(meta_compound, jump_moa.dropna(subset=['InChIKey']), left_on=['Metadata_InChIKey'], right_on=['InChIKey'], how='inner').groupby('Metadata_InChIKey').size().sort_values(ascending=False)

for inchi_key, count in counts.items():
    print(f"{inchi_key}: {count}")

In [ ]:
df_jump_moa = pd.merge(meta_compound, jump_moa.dropna(subset=['InChIKey']), left_on=['Metadata_InChIKey'], right_on=['InChIKey'], how='inner')

In [ ]:
df_jump_moa.moa.unique()

In [ ]:
df_jump_moa.groupby('moa').size().sort_values(ascending=False)[df_jump_moa.groupby('moa').size().sort_values(ascending=False)>100].index.values

In [ ]:
df_jump_moa_selected = df_jump_moa[df_jump_moa.moa.isin(df_jump_moa.groupby('moa').size().sort_values(ascending=False)[df_jump_moa.groupby('moa').size().sort_values(ascending=False)>100].index.values)].groupby('Metadata_InChIKey').size().sort_values(ascending=False).index.values

In [ ]:
df_jump_moa_selected

In [ ]:
df_jump_moa[df_jump_moa.Metadata_InChIKey.isin(df_jump_moa_selected)]

## jump_target1_compound

In [ ]:
counts = pd.merge(meta_compound, jump_target1_compound.dropna(subset=['InChIKey']), left_on=['Metadata_InChIKey'], right_on=['InChIKey'], how='inner').groupby('Metadata_InChIKey').size().sort_values(ascending=False)

for inchi_key, count in counts.items():
    print(f"{inchi_key}: {count}")

In [ ]:
df_jump_target1_compound = pd.merge(meta_compound, jump_target1_compound.dropna(subset=['InChIKey']), left_on=['Metadata_InChIKey'], right_on=['InChIKey'], how='inner')

In [ ]:
df_jump_target1_compound.control_type.unique()

In [ ]:
df_jump_target1_compound[df_jump_target1_compound.control_type.isin(['negcon','poscon_diverse', 'poscon_cp', 'poscon_orf'])].groupby('Metadata_InChIKey').size().sort_values(ascending=False)

In [ ]:
mask = df_jump_target1_compound[df_jump_target1_compound.control_type.isin(['negcon','poscon_diverse', 'poscon_cp', 'poscon_orf'])].groupby('Metadata_InChIKey').size().sort_values(ascending=False)>100

In [ ]:
df_jump_target1_compound_selected = df_jump_target1_compound[df_jump_target1_compound.control_type.isin(['negcon','poscon_diverse', 'poscon_cp', 'poscon_orf'])].groupby('Metadata_InChIKey').size().sort_values(ascending=False).index.values[mask]

In [ ]:
df_jump_target1_compound_selected

In [ ]:
df_jump_target1_compound[df_jump_target1_compound.Metadata_InChIKey.isin(df_jump_target1_compound_selected)]

## jump_target1_orf

In [ ]:
counts = pd.merge(meta_orf,jump_target1_orf.dropna(subset=['broad_sample']),left_on=['Metadata_broad_sample'],right_on=['broad_sample'],how='inner').groupby('Metadata_broad_sample').size().sort_values(ascending=False)

for inchi_key, count in counts.items():
    print(f"{inchi_key}: {count}")

In [ ]:
df_jump_target1_orf = pd.merge(meta_orf,jump_target1_orf.dropna(subset=['broad_sample']),left_on=['Metadata_broad_sample'],right_on=['broad_sample'],how='inner')

In [ ]:
df_jump_target1_orf.control_type.unique()

In [ ]:
df_jump_target1_orf[df_jump_target1_orf.control_type.isin(['poscon_diverse', 'negcon', 'poscon_cp', 'poscon_orf'])].groupby('Metadata_broad_sample').size().sort_values(ascending=False)

In [ ]:
mask = df_jump_target1_orf[df_jump_target1_orf.control_type.isin(['poscon_diverse', 'negcon', 'poscon_cp', 'poscon_orf'])].groupby('Metadata_broad_sample').size().sort_values(ascending=False)>100

In [ ]:
df_jump_target1_orf_selected = df_jump_target1_orf[df_jump_target1_orf.control_type.isin(['poscon_diverse', 'negcon', 'poscon_cp', 'poscon_orf'])].groupby('Metadata_broad_sample').size().sort_values(ascending=False).index.values[mask]

In [ ]:
df_jump_target1_orf_selected

In [ ]:
df_jump_target1_orf[df_jump_target1_orf.Metadata_broad_sample.isin(df_jump_target1_orf_selected)]

## jump_target2

In [ ]:
counts = pd.merge(meta_compound,jump_target2.dropna(subset=['InChIKey']),left_on=['Metadata_InChIKey'],right_on=['InChIKey'],how='inner').groupby('Metadata_InChIKey').size().sort_values(ascending=False)
dict = {}
for inchi_key, count in counts.items():
    dict[inchi_key] = count
    print(f"{inchi_key}: {count}")

In [ ]:
dict['MTJHLONVHHPNSI-UHFFFAOYSA-N']

In [ ]:
df_jump_target2 = pd.merge(meta_compound,jump_target2.dropna(subset=['InChIKey']),left_on=['Metadata_InChIKey'],right_on=['InChIKey'],how='inner')

In [ ]:
df_jump_target2.control_type.unique()

In [ ]:
df_jump_target2[df_jump_target2.control_type.isin(['poscon_diverse', 'negcon', 'poscon_cp', 'poscon_orf'])].groupby('Metadata_InChIKey').size().sort_values(ascending=False)

In [ ]:
mask = df_jump_target2[df_jump_target2.control_type.isin(['poscon_diverse', 'negcon', 'poscon_cp', 'poscon_orf'])].groupby('Metadata_InChIKey').size().sort_values(ascending=False)>100

In [ ]:
df_jump_target2_selected = df_jump_target2[df_jump_target2.control_type.isin(['poscon_diverse', 'negcon', 'poscon_cp', 'poscon_orf'])].groupby('Metadata_InChIKey').size().sort_values(ascending=False).index.values[mask]

In [ ]:
df_jump_target2[df_jump_target2.Metadata_InChIKey.isin(df_jump_target2_selected)]

## Retrieving Compounds name and df

In [ ]:
import re

In [ ]:
sub_df_jump_moa = df_jump_moa[df_jump_moa.Metadata_InChIKey.isin(df_jump_moa_selected)]
sub_df_jump_target1_compound = df_jump_target1_compound[df_jump_target1_compound.Metadata_InChIKey.isin(df_jump_target1_compound_selected)]
sub_df_jump_target2 = df_jump_target2[df_jump_target2.Metadata_InChIKey.isin(df_jump_target2_selected)]

In [ ]:
def add_metadata_prefix(df):
    """
    Adds the prefix 'Metadata_' to the column names of a DataFrame that do not already have it.
    
    Args:
        df (pandas.DataFrame): The input DataFrame.
        
    Returns:
        pandas.DataFrame: A new DataFrame with the column names updated.
    """
    # Get the list of columns that do not start with 'Metadata_'
    cols_to_rename = [col for col in df.columns if not col.startswith('Metadata_')]
    
    # Create a dictionary mapping the old column names to the new ones
    rename_dict = {col: 'Metadata_' + col for col in cols_to_rename}
    
    # Rename the columns using the dictionary
    df_renamed = df.rename(columns=rename_dict)
    
    return df_renamed

In [ ]:
sub_df_jump_moa = add_metadata_prefix(sub_df_jump_moa)
sub_df_jump_target1_compound = add_metadata_prefix(sub_df_jump_target1_compound)
sub_df_jump_target2 = add_metadata_prefix(sub_df_jump_target2)

In [ ]:
sub_df_jump_moa.columns

In [ ]:
sub_df_jump_target1_compound.columns

In [ ]:
sub_df_jump_target2.columns

In [ ]:
data_interest = pd.concat([sub_df_jump_target1_compound,sub_df_jump_target2])

In [ ]:
data_interest

In [ ]:
data_interest['Metadata_moa'] = np.nan

In [ ]:
def make_unique_columns(columns):
    seen = {}
    new_columns = []
    for col in columns:
        if col in seen:
            seen[col] += 1
            new_columns.append(f"{col}{seen[col]}")
        else:
            seen[col] = 0
            new_columns.append(col)
    return new_columns

In [ ]:
def remove_numbered_columns(df):
    # Create a list of column names without numbers
    clean_columns = [col for col in df.columns if not re.search(r'\d', col)]
    
    # Return a new DataFrame with only the clean columns
    return df[clean_columns]

In [ ]:
data_interest.columns = make_unique_columns(data_interest.columns)
data_interest = remove_numbered_columns(data_interest)

In [ ]:
sub_df_jump_moa.columns = make_unique_columns(sub_df_jump_moa.columns)
sub_df_jump_moa = remove_numbered_columns(sub_df_jump_moa)

In [ ]:
sub_df_jump_moa['Metadata_target'] = np.nan

In [ ]:
sub_df_jump_moa = sub_df_jump_moa.reindex(columns=data_interest.columns)

In [ ]:
data_interest = pd.concat([data_interest,sub_df_jump_moa])

# Final DF

In [ ]:
ewrwerewr

In [ ]:
data_interest

In [ ]:
data_interest.to_csv('../data/metadata/metadata_compound_of_interest.csv',index=False)

In [ ]:
df_load_check = pd.read_csv('../data/metadata/metadata_compound_of_interest.csv', dtype=str)

In [ ]:
df_load_check

In [ ]:
df_load_check.groupby('Metadata_control_type').size()

In [ ]:
df_load_check[df_load_check.Metadata_control_type=='poscon_diverse']

In [ ]:
df_load_check[df_load_check.Metadata_control_type=='poscon_diverse'].Metadata_InChIKey.unique()

In [ ]:
df_load_check[df_load_check.Metadata_control_type=='poscon_diverse'].Metadata_target.unique()

In [ ]:
df_load_check[(df_load_check.Metadata_control_type=='poscon_diverse') & (df_load_check.Metadata_target=='CHEK2')].Metadata_InChIKey.unique()

In [ ]:
jump_target1_compound[jump_target1_compound.control_type=='poscon_diverse'].InChIKey.unique()

In [ ]:
jump_target1_compound[jump_target1_compound.control_type=='poscon_diverse'].target.unique()

In [ ]:
jump_target1_compound[(jump_target1_compound.control_type=='poscon_diverse') & (jump_target1_compound.target=='PLK1')].InChIKey.unique()

# Data Selection

In [ ]:
df_load_check

In [ ]:
df_load_check.Metadata_control_type.unique()

In [ ]:
poscon_cp_InChIKey = df_load_check[df_load_check.Metadata_control_type == 'poscon_cp'].Metadata_InChIKey.unique()

In [ ]:
np.savetxt("../data/metadata/poscon_cp_InChIKey.txt", poscon_cp_InChIKey, fmt="%s")

In [ ]:
known_moa_InChIKey = df_load_check[~df_load_check.Metadata_moa.isna()].Metadata_InChIKey.unique()

In [ ]:
np.savetxt("../data/metadata/known_moa_InChIKey.txt", known_moa_InChIKey, fmt="%s")

In [ ]:
len(poscon_cp_InChIKey) + len(known_moa_InChIKey)

In [ ]:
len(list(set(list(poscon_cp_InChIKey)+list(known_moa_InChIKey))))